In [ ]:
%store -r
import time
import os,json
import requests
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitoralert
from dkube.sdk.rsrcs.modelmonitor import DataType, ChannelOrder, ImageDataSavedFileFormat

In [ ]:
api = DkubeApi(URL=os.getenv('DKUBE_URL'),token=os.getenv("DKUBE_USER_ACCESS_TOKEN"))

In [ ]:
deployment_id = api.get_deployment_id(name=MONITOR_NAME)

In [ ]:
mm=DkubeModelmonitor(name=MONITOR_NAME, model_type=ModelType.Classification.value)

In [ ]:
mm.update_modelmonitor(data_timezone="utc", input_data_type=data_type = DataType.Image.value)

In [ ]:
mm.update_image_data_shape(height=200, width=200)

In [ ]:
training_data = f'{DKUBE_BASE_DATASET}:'+ DKUBEUSERNAME
train_data_version = 'v1:'+api.get_dataset_versions(
    DKUBEUSERNAME,
    DKUBE_BASE_DATASET)[0]['version']['uuid']
prediction_data = LIVE_DATASET + ":"+ DKUBEUSERNAME
labelled_data = LIVE_DATASET + ":"+ DKUBEUSERNAME
predict_data_format = "cloudeventlogs"

In [ ]:
training_data, prediction_data, labelled_data

In [ ]:
mm.add_datasources(data_class=str(DatasetClass.Train),name=training_data,
                   version=train_data_version)
mm.add_datasources(data_class=str(DatasetClass.Predict),name=prediction_data,
                   s3_subpath="", data_format=predict_data_format,
                   date_suffix="yyyy/mm/dd/hh")

In [ ]:
mm.update_drift_monitoring_details(enabled=True,
                                  frequency=5,algorithm='auto',
                                  image_train_data_savedfile_format="images_in_labelled_folder",
                                  image_predict_data_savedfile_format="images_in_labelled_folder")

## Health Monitoring

In [ ]:
deployment = api.get_deployment(deployment_id)
if deployment.data.imported_deployment and deployment.data.imported_deployment.cluster:
    mm.update_deployment_monitoring_details(enabled=True, frequency=1, collect_metrics=True)

In [ ]:
api.modelmonitor_create(deployment_id, mm,wait_for_completion=True)

### Start the model monitor

In [ ]:
api.modelmonitor_start(deployment_id)

### Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(id)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(id)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(id)
    